<a href="https://www.kaggle.com/code/jhtkoo0426/house-price-predictions-ft-catboostregressor?scriptVersionId=150685333" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

In [1]:
import numpy as np
import pandas as pd

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Create pipeline
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Models
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostRegressor

# Load datasets

In [2]:
train_ds = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
test_ds  = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [3]:
X = train_ds.drop(columns=['Id', 'SalePrice'], axis=1)
y = train_ds['SalePrice']

# Exploring the dataset

## Dataset information
First, let's see what data types the dataset consists of.

In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

From the dataset information, we observe that there are:
- 36 columns containing **numerical** data; and
- 43 columns containing **categorical or object** data

Since the handling of these two types of data differs, we will segregate them accordingly.

In [5]:
numeric_features = X.select_dtypes(include=['number'])
categorical_features = X.select_dtypes(include=['object'])
categorical_features.columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [6]:
sns.heatmap(numeric_features.corr())

<Axes: >

## Null columns
Let's take a look at which columns contain `NULL` values. This will provide us a reference point when we construct a pipeline class to process `NULL` values.

In [7]:
feature_contains_null = X.isna().sum().sort_values(ascending=False)
feature_contains_null[feature_contains_null > 0]

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
MasVnrType       872
FireplaceQu      690
LotFrontage      259
GarageCond        81
GarageType        81
GarageYrBlt       81
GarageQual        81
GarageFinish      81
BsmtFinType2      38
BsmtExposure      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
Electrical         1
dtype: int64

# Data preprocessing
Deal with `NULL` values according to the data description and treating special cases.

In [8]:
class NullProcessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Fill categorical NA values based on data description
        for col in ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'Fence', 'MiscFeature']:
            X[col] = X[col].fillna('NA')
        
        for col in ['MasVnrType']:
            X[col] = X[col].fillna('None')
        
        for col in ['MasVnrArea', 'LotFrontage']:
            X[col] = X[col].fillna(0)
            
        # Special cases - we want to fill NULL values in these columns with special values instead.
        X['Electrical'] = X['Electrical'].fillna('SBrkr')
        return X

# Feature Engineering

In [9]:
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X['AvgRoomArea'] = X['GrLivArea'] / (X['TotRmsAbvGrd'] + X['FullBath'] + X['HalfBath'] + X['KitchenAbvGr'])
        X['BathroomCnt'] = X['FullBath'] + (0.5 * X['HalfBath']) + X['BsmtFullBath'] + (0.5 * X['BsmtHalfBath'])
        X["TotalArea"] = X["GrLivArea"] + X["1stFlrSF"] + X["2ndFlrSF"] + X["GarageArea"] + X["TotalBsmtSF"] + X["MasVnrArea"]
        X["Age"] = pd.to_numeric(X["YrSold"]) - pd.to_numeric(X["YearBuilt"])
        X["Renovate"] = pd.to_numeric(X["YearRemodAdd"]) - pd.to_numeric(X["YearBuilt"])
        return X

# Create pipeline

In [10]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('column_dropper', 'drop', ['PoolQC', 'GarageYrBlt']),
    ('num', numeric_transformer, numeric_features.columns),
    ('cat', categorical_transformer, categorical_features.columns),
])

Combine preprocessing with the model in a single pipeline

In [11]:
pipeline = Pipeline(steps=[
    ('null_processor', NullProcessor()),
    ('feature_engineer', FeatureEngineer()),
    ('preprocessor', preprocessor),
    ('classifier', CatBoostRegressor()),
])

Fit the pipeline on the training data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
pipeline.fit(X_train, y_train)

Learning rate set to 0.04196
0:	learn: 76281.3854456	total: 61.9ms	remaining: 1m 1s
1:	learn: 74479.6115399	total: 69.8ms	remaining: 34.8s
2:	learn: 72564.9961638	total: 74.3ms	remaining: 24.7s
3:	learn: 70808.1050432	total: 79.2ms	remaining: 19.7s
4:	learn: 69073.1598807	total: 83.9ms	remaining: 16.7s
5:	learn: 67467.8943152	total: 88.5ms	remaining: 14.7s
6:	learn: 65892.9376416	total: 93.2ms	remaining: 13.2s
7:	learn: 64356.5787582	total: 97.6ms	remaining: 12.1s
8:	learn: 62796.8579653	total: 102ms	remaining: 11.3s
9:	learn: 61383.3888925	total: 106ms	remaining: 10.5s
10:	learn: 59935.5254468	total: 111ms	remaining: 9.97s
11:	learn: 58688.4257524	total: 115ms	remaining: 9.49s
12:	learn: 57575.7563329	total: 119ms	remaining: 9.06s
13:	learn: 56203.2924498	total: 124ms	remaining: 8.71s
14:	learn: 54926.4840720	total: 128ms	remaining: 8.41s
15:	learn: 53824.7094290	total: 132ms	remaining: 8.15s
16:	learn: 52799.9931733	total: 137ms	remaining: 7.92s
17:	learn: 51775.8856905	total: 142ms	

Pipeline(steps=[('null_processor', NullProcessor()),
                ('feature_engineer', FeatureEngineer()),
                ('preprocessor',
                 ColumnTransformer(transformers=[('column_dropper', 'drop',
                                                  ['PoolQC', 'GarageYrBlt']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBui...
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))])),
                ('classifier',
                 <catboost.core.CatBoostRegressor object at 0x799cb445dff0>)])

In [14]:
train_predictions = pipeline.predict(X_test)

In [15]:
rmse_score = np.sqrt(mean_squared_error(train_predictions, y_test))
rmse_score

24905.36585805831

# Making predictions & submission file

Make predictions on the testing data

In [16]:
submission_df = test_ds[['Id']]
test_ds = test_ds.drop(columns=['Id'])

In [17]:
predicted_prices = pipeline.predict(test_ds)

In [18]:
submission_df['SalePrice'] = predicted_prices
submission_df = submission_df.set_index('Id')
submission = submission_df.to_csv("submission.csv", encoding='utf-8')

In [19]:
!head submission.csv

Id,SalePrice
1461,126236.766761648
1462,158566.91559989154
1463,191678.95751514027
1464,192709.84164007465
1465,183739.89082745695
1466,178231.03950192576
1467,172398.49983720615
1468,167303.94985575028
1469,184513.83066095633
